In [1]:
import os
import mlflow
import optuna
import joblib
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from pathlib import Path
from dotenv import load_dotenv
import sys

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from feature_engine.imputation import DropMissingData
from feature_engine.selection import DropFeatures
from feature_engine.outliers import ArbitraryOutlierCapper
from feature_engine.timeseries.forecasting import WindowFeatures
import matplotlib
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['lines.markersize'] = 8.5

/home/marcos/miniconda3/envs/vfm_3wdataset/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **3W Data**

The 3W dataset is a collection of sensor measurements gathered from real-world offshore oil well operations, supplemented by software-simulated data and hand-crafted synthetic data. It includes eight types of abnormal events.

The data is organized into separate folders for each type of event, with each individual data acquisition session saved as a comma-separated value (CSV) file. These sessions capture sensor measurements recorded over time. The files are named according to their data source (real, simulated, or synthetic), followed by sequential numbering. For real events, a timestamp is also included in the file name.

Each file represents an instance of a type of abnormal event and typically contains the entire signal, starting from normal operation, progressing through the transient period, and concluding with the steady-state anomaly. The following table provides a summary of the number of instances available for each abnormal event and its source:

<center><img src="../figures/instances_3wdataset.png" width="600" height="400"></center>


The following table shows the number of observations (or data points) captured during real events in the dataset. Each event is categorized into three stages: Normal, Transient, and Steady-state.

- **Normal**: Period when the system operates without any anomalies.
- **Transient**:  Phase where the anomaly begins to develop.
- **Steady-state**: Phase where the anomaly has fully manifested and stabilized.


<center><img src="../figures/nobservations_real_3wdataset.png" width="600" height="400"></center>

## **Observations**

1) From tables 1 and 2, we can see that the flow instability event is only represented by real instances and is available just for steady-state signal. In flow instability, the boundaries between normal and abnormal conditions are not clearly defined because the system slowly moves from a stable state to an unstable one. These small changes make it difficult to determine exactly when the anomaly starts.

2) From tables 1 and 2, we can see that the Severe Slugging event has data from both simulated and real sources, but for real data, there is only for the steady-state. This is because Severe Slugging typically has a sudden onset, meaning the transition from normal operation to slugging occurs abruptly without a gradual or easily detectable progression. As a result, the dataset captures only the fully developed, steady-state phase of the fault, where the severe slugging has stabilized and is most prominent. The rapid nature of this event makes it difficult to capture any transient period before the fault fully manifests.

3) The dataset's class imbalance, where certain events have far fewer instances or observations. This imbalance poses challenges for machine learning, as it can lead to biased models. Therefore, model evaluation requires special consideration, particularly in selecting appropriate metrics like precision, recall, or F1-score to accurately assess performance in the imbalanced data.




## **Eight  types of abnormal events (Classes):** 

0) **Normal**:
Naturally flowing wells are those in which the formation pressure is sufficient to produce oil commercially without requiring a pump. Most reservoirs at the initial stage of development have enough pressure for a natural flow and thus require less equipment and automation for control and successful oil and gas production.

1) **Abrupt increase in BSW (Basic Sediment and Water)**:
Suspended water, sediments and other impurities in the production measured as a percentage of the production stream.The lifecycle of each well contains periods of increasing levels of BSW. However, an unexpected rise indicates a developing production issue, which needs to be remedied quickly.

2) **Spurious closure of the DHSV (Downhole Safety Valve)**:
The valve isolates wellbore fluids in a catastrophic failure of surface equipment. If the valve fails spuriously without any surface signs, it needs to be reopened; hence, an automatic event identification is essential.

3) **Severe slugging**:
An event in which large gas bubbles follow a sequence of liquid slugs. It is a cyclical phenomenon that can lead to wellhead and pipeline damage. Hence, it is considered a critical abnormality.

4) **Flow instability**:
Pressure changes within acceptable thresholds, with differences due to slugging, representing the absence of cyclicity. This event can transform into slugging and then a severe variant, which requires imminent actions. 

5) **Rapid productivity decline**: 
Flow loss due to changes in reservoir static pressure, alternating BSW percentage, production viscosity, changes in production line diameter, etc.

6) **Sudden restriction in the production choke (PCK)**: 
Quick restriction in the production choke (PCK) – a term Petrobras uses to indicate issues with a PCK valve installed at the beginning of the production line. Short restrictions might be observed when operated manually due to operational problems that need to be identified and reversed.

7) **Scaling in the PCK**:
A mineral deposit, which can create a significant restriction or even a plug in the production tubing. Thus, monitoring production choke helps recognise the event and take appropriate actions, such as scale inhibitor injections.

8) **Hydrate formation in the production line**:
Compounds of complex ions formed by water and other substances at reduced temperatures and high pressure, which might lead to plugging of the pipelines. It is one of the most significant issues in oil and gas production, and it can stop flow for an extended period; hence, it needs to be recognised immediately.




## **Measurements:**

**PDG(Permanent Downhole Gauge)**
A sensor permanently installed in an oil or gas well, designed to continuously monitor real-time conditions.

**TPT(Temperature and Pressure Transducer)**
A sensor capable of measuring both temperature and pressure in the system. Located inside the Subsea Christmas Tree

**MON-CKP(Upstream Pressure and Temperature Sensor at Production Choke)**
A sensor that measures the pressure and temperature of the fluid upstream, or before it flows through the production choke (PCK), a valve used to regulate the flow rate of oil or gas.

Note: CKP refers to the production choke, which is typically located on the platform.

**JUS-CKP(production choke)**
A sensor that measures the temperature of the fluid downstream (after) it has passed through the Production Choke (PCK). The data from this sensor helps assess how pressure changes across the choke affect the temperature of the fluid

**JUS-CKGL(Downstream Temperature Sensor at Production Choke)**
A sensor that measures the temperature of the fluid downstream, or after it has passed through the production choke (PCK). This data is important for analyzing how pressure reduction across the choke affects fluid temperature and flow characteristics.

**QGL(Gas Lift Flow Rate)**
Measurement of the flow rate of gas being injected into the well through the gas lift system. Gas lift is a technique used to enhance oil production by reducing the density of the fluid column and enabling easier flow from the reservoir.




## **Features predicting Severe Slugging in binary classification:**


- **P-PDG**: Temperature at the PDG (Permanent Downhole Gauge) [ºC];

- **P-TPT**: Pressure at the TPT (temperature and pressure transducer) [Pa];

- **T-TPT**: Temperature at the TPT (temperature and pressure transducer) [ºC];

- **P-MON-CKP**: Upstream pressure of the PCK (production choke) [Pa];

- **T-JUS-CKP**: Downstream temperature of the PCK (production choke) [ºC];

- **P-JUS-CKGL**: Downstream pressure of the GLCK (gas lift choke) [Pa];

- **T-JUS-CKGL**: Downstream temperature of the GLCK (gas lift choke) [ºC];

- **QGL**: Gas lift flow rate [m3/s];

- **class**: Label of the observation;

- **id**: instance identifier. Hand-drawn and simulated instances have incremental id. Each real instance has an id generated from its first timestamp.

### **Boolean columns to check if the value is missing**

- P-PDG__is_missing
- P-TPT__is_missing
- T-TPT__is_missing
- P-MON-CKP__is_missing
- T-JUS-CKP__is_missing       
- P-JUS-CKGL__is_missing
- T-JUS-CKGL__is_missing
- QGL__is_missing

# **Binary Classification for Severe Slugging**

# Load metadata

In [27]:
def load_metadata(download_dir, positive_event='3'):
    addresses, instances, events, y = [], [], [], []
    
    for root, dirs, files in os.walk(download_dir):
        for file in files:
            if file.endswith('.csv'):
                address = os.path.join(root, file)  # Full file path
                instance = file  # The file name itself is the instance
                event = root.split('/')[-1]  # Event is the directory (e.g., '0', '3', etc.)
                y_ = '1' if event == positive_event else '0'
                
                addresses.append(address)
                instances.append(instance)
                events.append(event)
                y.append(y_)
    sorted_data = sorted(zip(addresses, instances, events, y), key=lambda x: x[0])
    addresses, instances, events, y = zip(*sorted_data)
    
    return list(addresses), list(instances), list(events), list(y)

download_dir = os.path.expanduser('~/GitHub/vfm_3wdataset/data/public')
addresses, instances, events, y = load_metadata(download_dir, positive_event='3')

In [28]:
download_dir = os.path.expanduser('~/GitHub/vfm_3wdataset/data/public')
addresses, instances, events, y = load_metadata(download_dir, positive_event='3')

addresses_train, addresses_test, instances_train, \
instances_test, events_train, events_test,\
y_train, y_test = train_test_split(
    addresses, instances, events, y,
    stratify=y,
    test_size=0.2,
    random_state=42
)

data_instances_info = np.column_stack(
    [
        np.concatenate([addresses_train, addresses_test], axis=0),
        np.concatenate([instances_train, instances_test], axis=0),
        np.concatenate([events_train, events_test], axis=0),
        np.concatenate([y_train, y_test], axis=0),
        np.concatenate([['Train']*len(y_train), ['Test']*len(y_test)])
    ]
)

df_metadata = pd.DataFrame(data=data_instances_info, columns=['Address', 'Instance', 'Event', 'Class', 'Set'])
df_metadata['id'] = np.arange(df_metadata.shape[0])
df_metadata.head()

,Address,Instance,Event,Class,Set,id
0,/home/marcos/GitHub/vfm_3wdataset/data/public/...,SIMULATED_00067.csv,3,1,Train,0
1,/home/marcos/GitHub/vfm_3wdataset/data/public/...,SIMULATED_00041.csv,3,1,Train,1
2,/home/marcos/GitHub/vfm_3wdataset/data/public/...,WELL-00005_20170817170000.csv,0,0,Train,2
3,/home/marcos/GitHub/vfm_3wdataset/data/public/...,SIMULATED_00039.csv,3,1,Train,3
4,/home/marcos/GitHub/vfm_3wdataset/data/public/...,WELL-00005_20170402230127.csv,0,0,Train,4


# Preprocessing data

In [29]:
def load_data(
    local_path: str,  
    positive_event: str = '3', 
    periods_undersample: int or str or None = None):

    df = pd.read_csv(local_path)
    
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df.index = df['timestamp']  
    df = df.drop(columns=['timestamp'])  
        
    df['class'] = df['class'].apply(lambda x: positive_event in str(x)).astype(int)

    df = df.ffill().bfill()

    for col in df.columns:
        if col != 'class': 
            df[f"{col}__is_missing"] = 1 if df.isna().sum()[col] == df.shape[0] else 0

    df = df.fillna(0)

    if type(periods_undersample) in [int, str]:
        if type(periods_undersample) == int:
            periods_undersample = f'{periods_undersample}s'  
            
        df = df.resample(periods_undersample).last() 
    
    return df

In [32]:
def load_all_data(
    addresses, instances, 
    events, ids,
    positive_event: str = '3',
    periods_undersample: int or str or None = None
):
    df_lst = []

    for address, instance, event, id_ in zip(addresses, instances, events, ids):
        local_file_path = address  

        df_temp = load_data(
            local_path=local_file_path,
            positive_event=positive_event,
            periods_undersample=periods_undersample
        )
        
        df_temp['id'] = id_
        df_lst.append(df_temp)
    
    df = pd.concat(df_lst, axis=0)

    return df


In [31]:
ids_train = df_metadata[df_metadata.Set == 'Train'].id.values
ids_test = df_metadata[df_metadata.Set == 'Test'].id.values

mean_y_train = np.mean(np.array(y_train).astype(float))
print(mean_y_train)
mean_y_test = np.mean(np.array(y_test).astype(float))
print(mean_y_test)

0.1512455516014235
0.14893617021276595


In [33]:
df_train = load_all_data( 
    addresses=addresses_train, 
    instances=instances_train, 
    events=events_train, 
    ids=ids_train,
    positive_event='3',
    periods_undersample=60
)
display(df_train.head())

df_test = load_all_data( 
    addresses=addresses_test, 
    instances=instances_test, 
    events=events, 
    ids=ids_test,
    positive_event='3',
    periods_undersample=60 
)
display(df_test.head())

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,P-PDG__is_missing,P-TPT__is_missing,T-TPT__is_missing,P-MON-CKP__is_missing,T-JUS-CKP__is_missing,P-JUS-CKGL__is_missing,T-JUS-CKGL__is_missing,QGL__is_missing,id
timestamp,,,,,,,,,,,,,,,,,,
2018-04-10 23:49:00,24048050.0,15656440.0,119.3033,10062140.0,90.57189,0.0,0.0,0.0,1,0,0,0,0,0,1,1,1,0
2018-04-10 23:50:00,24108100.0,15736100.0,119.2536,10066590.0,90.86570,0.0,0.0,0.0,1,0,0,0,0,0,1,1,1,0
2018-04-10 23:51:00,24147830.0,15796550.0,119.2263,10070230.0,91.16160,0.0,0.0,0.0,1,0,0,0,0,0,1,1,1,0
2018-04-10 23:52:00,24169110.0,15839570.0,119.2203,10072970.0,91.44675,0.0,0.0,0.0,1,0,0,0,0,0,1,1,1,0
2018-04-10 23:53:00,24173070.0,15865880.0,119.2369,10074940.0,91.71573,0.0,0.0,0.0,1,0,0,0,0,0,1,1,1,0


,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,P-PDG__is_missing,P-TPT__is_missing,T-TPT__is_missing,P-MON-CKP__is_missing,T-JUS-CKP__is_missing,P-JUS-CKGL__is_missing,T-JUS-CKGL__is_missing,QGL__is_missing,id
timestamp,,,,,,,,,,,,,,,,,,
2013-12-12 07:00:00,0.0,16647840.0,118.0997,7360490.0,173.0961,3931823.0,0.0,0.0,0,0,0,0,0,0,0,1,0,562
2013-12-12 07:01:00,0.0,16643050.0,118.1049,7356511.0,173.0961,3933994.0,0.0,0.0,0,0,0,0,0,0,0,1,0,562
2013-12-12 07:02:00,0.0,16638260.0,118.0912,7352533.0,173.0961,3936164.0,0.0,0.0,0,0,0,0,0,0,0,1,0,562
2013-12-12 07:03:00,0.0,16644150.0,118.1107,7348554.0,173.0961,3938334.0,0.0,0.0,0,0,0,0,0,0,0,1,0,562
2013-12-12 07:04:00,0.0,16650800.0,118.1049,7435900.0,173.0961,3940504.0,0.0,0.0,0,0,0,0,0,0,0,1,0,562


In [35]:
df_test['class'].unique()

array([0, 1])

In [ ]:
class

# Analysis

https://glossary.slb.com/en/search#q=bsw&sort=relevancy

create hyperlink 


